In [34]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
import os 
import shutil
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt 
import torch
import cv2
import pytesseract as pt 
%matplotlib inline

In [6]:
!pip install pyyaml

In [3]:
!pip install scikit-learn

  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)


In [1]:
!pip install pytesseract

In [11]:
filenames = []

size_props = {
    'height':[],
    'width':[]
}

bounding_box_props = {
    'xmin':[],
    'ymin':[],
    'xmax':[],
    'ymax':[]
}

In [12]:
annotations_path = 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\Annotations'
for file in tqdm(os.listdir(annotations_path)):
    annotation = ET.parse(os.path.join(annotations_path, file))
    filenames.append(os.path.join(annotations_path, file))
    size = annotation.find('size')
    for name, prop_list in size_props.items():
        prop_value = size.find(name).text
        size_props[name].append(int(prop_value))
    bounding_box = annotation.find('object').find('bndbox')
    for name, prop_list in bounding_box_props.items():
        prop_value = bounding_box.find(name).text
        bounding_box_props[name].append(int(prop_value))

100%|██████████| 669/669 [00:00<00:00, 5528.31it/s]


In [13]:
df = pd.DataFrame({
    'file':filenames,
    'width':size_props['width'],
    'height':size_props['height'],
    'xmin':bounding_box_props['xmin'],
    'ymin':bounding_box_props['ymin'],
    'xmax':bounding_box_props['xmax'],
    'ymax':bounding_box_props['ymax']
})

In [14]:
df.head()

,file,width,height,xmin,ymin,xmax,ymax
0,C:\Users\Rupesh\Desktop\New_project\ANPR_Assig...,806,466,581,273,700,320
1,C:\Users\Rupesh\Desktop\New_project\ANPR_Assig...,494,449,79,381,288,449
2,C:\Users\Rupesh\Desktop\New_project\ANPR_Assig...,642,400,199,138,421,219
3,C:\Users\Rupesh\Desktop\New_project\ANPR_Assig...,248,177,28,52,227,161
4,C:\Users\Rupesh\Desktop\New_project\ANPR_Assig...,248,177,20,55,226,153


In [15]:
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']

In [16]:
df.head()

,file,width,height,xmin,ymin,...,ymax,center_x,center_y,bb_width,bb_height
0,C:\Users\Rupesh\Desktop\New_project\ANPR_Assig...,806,466,581,273,...,320,0.794665,0.636266,0.147643,0.100858
1,C:\Users\Rupesh\Desktop\New_project\ANPR_Assig...,494,449,79,381,...,449,0.371457,0.924276,0.423077,0.151448
2,C:\Users\Rupesh\Desktop\New_project\ANPR_Assig...,642,400,199,138,...,219,0.482866,0.446250,0.345794,0.202500
3,C:\Users\Rupesh\Desktop\New_project\ANPR_Assig...,248,177,28,52,...,161,0.514113,0.601695,0.802419,0.615819
4,C:\Users\Rupesh\Desktop\New_project\ANPR_Assig...,248,177,20,55,...,153,0.495968,0.587571,0.830645,0.553672


In [17]:
# Keeping important columns only 
yolo_df = df[['file', 'center_x', 'center_y', 'bb_width', 'bb_height']]
# Performing 70-15-15 split
test_size = int(0.15 * len(df))

df_train, df_test = train_test_split(yolo_df, test_size=test_size)
df_train, df_val = train_test_split(df_train, test_size=test_size)

In [18]:
train_path = os.path.join('Images', 'train')
val_path = os.path.join('Images','val')
test_path = os.path.join('Images', 'test')
images_path = 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages'

if not os.path.exists(train_path):
    os.makedirs(train_path)
    print('Made folder for train set')

if not os.path.exists(val_path):
    os.makedirs(val_path)
    print('Made folder for val set')

if not os.path.exists(test_path):
    os.makedirs(test_path)
    print('Made folder for test set')

In [19]:
print('Moving images for train set')
for _, row  in tqdm(df_train.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.jpg')
    image_dst = os.path.join(train_path, f'{image_name}.jpg')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(train_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for train set')

print('Moving images for val set')
for _, row  in tqdm(df_val.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.jpg')
    image_dst = os.path.join(val_path, f'{image_name}.jpg')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(val_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for val set')

print('Moving images for test set')
for _, row  in tqdm(df_test.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.jpg')
    image_dst = os.path.join(test_path, f'{image_name}.jpg')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(test_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for test set')

Moving images for train set


469it [00:01, 433.63it/s]


Done moving images for train set
Moving images for val set


100it [00:00, 325.54it/s]


Done moving images for val set
Moving images for test set


100it [00:00, 374.14it/s]

Done moving images for test set


In [1]:
! git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...


In [12]:
!pip install -r yolov5/requirements.txt

  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached cachetools-5.2.0-py3-none-any.whl (9.3 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached importlib_metadata-4.12.0-py3-none-any.whl (21 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.2
    Uninstalling protobuf-4.21.2:
      Successfully uninstalled protobuf-4.21.2


In [20]:
data = {
    'names':['License Plate'],
    'nc':1,
    'train':os.path.abspath(train_path),
    'val':os.path.abspath(val_path)
}

with open('data.yaml', 'w') as f:
    yaml.dump(data, f)

In [14]:
# Training the YOLO model 
! python ./yolov5/train.py --data ./data.yaml  --batch-size 32  --epochs 15 --weights yolov5/yolov5s.pt

In [19]:
!pwd

/content


In [20]:
!ls

data.yaml  drive  Images  sample_data  yolov5


In [2]:
import torch

c:\Users\Rupesh\.conda\envs\ada_project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
# Fetching the latest runs
yolo_path = 'C:\\Users\\Rupesh\\Desktop\\yolo\\yolov5\\content\\yolov5\\runs\\train'
latest_run = os.listdir(yolo_path)[-1]

# Fetching the best weights 
best_weights = os.path.join(yolo_path, latest_run, 'weights', 'best.pt')

# Loading the model with best weights trained on custom data 
model = torch.hub.load('ultralytics/yolov5', 'custom', best_weights)

Using cache found in C:\Users\Rupesh/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-9-5 Python-3.9.12 torch-1.12.0 CUDA:0 (GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [30]:
!pip3 uninstall PIL
!pip3 uninstall Pillow
!pip3 install Pillow

^C


In [19]:
!ls '/content/Images/test'

car_102.jpg  car_199.jpg  car_309.jpg  car_385.jpg  car_501.jpg  car_601.jpg
car_102.txt  car_199.txt  car_309.txt  car_385.txt  car_501.txt  car_601.txt
car_111.jpg  car_201.jpg  car_320.jpg  car_396.jpg  car_511.jpg  car_603.jpg
car_111.txt  car_201.txt  car_320.txt  car_396.txt  car_511.txt  car_603.txt
car_116.jpg  car_202.jpg  car_325.jpg  car_398.jpg  car_513.jpg  car_609.jpg
car_116.txt  car_202.txt  car_325.txt  car_398.txt  car_513.txt  car_609.txt
car_121.jpg  car_210.jpg  car_32.jpg   car_400.jpg  car_528.jpg  car_612.jpg
car_121.txt  car_210.txt  car_32.txt   car_400.txt  car_528.txt  car_612.txt
car_133.jpg  car_211.jpg  car_330.jpg  car_418.jpg  car_531.jpg  car_614.jpg
car_133.txt  car_211.txt  car_330.txt  car_418.txt  car_531.txt  car_614.txt
car_136.jpg  car_224.jpg  car_334.jpg  car_432.jpg  car_536.jpg  car_617.jpg
car_136.txt  car_224.txt  car_334.txt  car_432.txt  car_536.txt  car_617.txt
car_144.jpg  car_240.jpg  car_335.jpg  car_435.jpg  car_539.jpg  car_625.jpg

In [22]:
df_test

,file,center_x,center_y,bb_width,bb_height
477,/content/drive/MyDrive/Colab Notebooks/Edge Ne...,0.356250,0.547619,0.126786,0.047619
48,/content/drive/MyDrive/Colab Notebooks/Edge Ne...,0.507500,0.686695,0.215000,0.094421
61,/content/drive/MyDrive/Colab Notebooks/Edge Ne...,0.303987,0.548673,0.279070,0.182891
460,/content/drive/MyDrive/Colab Notebooks/Edge Ne...,0.857500,0.781633,0.285000,0.159184
106,/content/drive/MyDrive/Colab Notebooks/Edge Ne...,0.366250,0.833333,0.267500,0.130081
...,...,...,...,...,...
418,/content/drive/MyDrive/Colab Notebooks/Edge Ne...,0.439000,0.732620,0.098000,0.042781
379,/content/drive/MyDrive/Colab Notebooks/Edge Ne...,0.503750,0.625000,0.132500,0.090000
71,/content/drive/MyDrive/Colab Notebooks/Edge Ne...,0.496250,0.556604,0.177500,0.119497
294,/content/drive/MyDrive/Colab Notebooks/Edge Ne...,0.511250,0.711321,0.227500,0.079245


In [43]:
test_files = df_test['file'].apply(lambda x: x.replace('Annotations','JPEGImages').replace('xml','jpg'))
test_images = np.random.choice(test_files, size=(5,5))

In [22]:
test_images

array([['C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_266.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_651.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_49.jpg',
        'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_564.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_210.jpg'],
       ['C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_595.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_124.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_579.jpg',
        'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_174.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_23.jpg'],
       ['C:\\Users\\Rupesh\\

In [45]:
test_images

array([['C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_210.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_514.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_415.jpg',
        'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_464.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_106.jpg'],
       ['C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_214.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_246.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_84.jpg',
        'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_370.jpg', 'C:\\Users\\Rupesh\\Desktop\\New_project\\ANPR_Assignment\\VOC2007\\JPEGImages\\car_664.jpg'],
       ['C:\\Users\\Rupesh\

In [51]:
!python ./yolov5/detect.py --source "C:\Users\Rupesh\Desktop\New_project\ANPR_Assignment\VOC2007\test_image" --conf 0.4 --weights ./yolov5/runs/train/exp/weights/best.pt --save-txt

detect: weights=['./yolov5/runs/train/exp/weights/best.pt'], source=C:\Users\Rupesh\Desktop\New_project\ANPR_Assignment\VOC2007\test_image, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.2-85-g9da6d0f Python-3.9.12 torch-1.12.0 CUDA:0 (GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/30 C:\Users\Rupesh\Desktop\New_project\ANPR_Assignment\VOC2007\test_image\car_12.jpg: 448x640 1 License Plate, 32.0ms
image 2/30 C:\Users\Rupesh\Desktop\New_project\ANPR_Assignment\VOC2007\test_image\car_13.jpg: 640x640 1 License Plate, 24.0ms
image 3/30 C:\Users\Rupesh\Desktop\Ne

In [52]:
import platform,socket,re,uuid,json,psutil,logging

def getSystemInfo():
    try:
        info={}
        info['platform']=platform.system()
        info['platform-release']=platform.release()
        info['platform-version']=platform.version()
        info['architecture']=platform.machine()
        info['hostname']=socket.gethostname()
        info['ip-address']=socket.gethostbyname(socket.gethostname())
        info['mac-address']=':'.join(re.findall('..', '%012x' % uuid.getnode()))
        info['processor']=platform.processor()
        info['ram']=str(round(psutil.virtual_memory().total / (1024.0 **3)))+" GB"
        return json.dumps(info)
    except Exception as e:
        logging.exception(e)

json.loads(getSystemInfo())

{'platform': 'Windows',
 'platform-release': '10',
 'platform-version': '10.0.22000',
 'architecture': 'AMD64',
 'hostname': 'LAPTOP-ILUMADRQ',
 'ip-address': '192.168.136.1',
 'mac-address': '60:a5:e2:ec:35:11',
 'processor': 'Intel64 Family 6 Model 165 Stepping 2, GenuineIntel',
 'ram': '8 GB'}